In [25]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
!pip install geocoder
!pip install folium
!pip install geopy

import requests
import pandas as pd
import geocoder
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import numpy as np
import geopy

from bs4 import BeautifulSoup as bsoup
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

print('Packages fully loaded and installed.')

Packages fully loaded and installed.


In [26]:
#define URL source
source = requests.get('https://en.wikipedia.org/wiki/Neighborhoods_in_Boston').text

#create a Beautiful Soup object and define lxml as the parser
soup= bsoup(source, 'lxml')
divlist = soup.find('div',class_ = 'div-col')
ulist = divlist.ul
ulist = ulist.text.split('\n')
ulist

['Allston',
 'Back Bay',
 'Bay Village',
 'Beacon Hill',
 'Brighton',
 'Charlestown',
 'Chinatown/Leather District',
 'Dorchester (divided for planning purposes into Mid Dorchester and Dorchester)',
 'Downtown',
 'East Boston',
 'Fenway Kenmore (includes Longwood)',
 'Hyde Park',
 'Jamaica Plain',
 'Mattapan',
 'Mission Hill',
 'North End',
 'Roslindale',
 'Roxbury',
 'South Boston',
 'South End',
 'West End',
 'West Roxbury']

In [27]:
df=pd.DataFrame(ulist)
df.columns = ['Neighborhood']
df.replace(['Dorchester (divided for planning purposes into Mid Dorchester and Dorchester)'],['Dorchester'], inplace=True)
df.replace(['Fenway Kenmore (includes Longwood)'], ['Fenway Kenmore'], inplace = True)
df.replace(['Chinatown/Leather District'],['Chinatown'], inplace=True)
df

,Neighborhood
0,Allston
1,Back Bay
2,Bay Village
3,Beacon Hill
4,Brighton
5,Charlestown
6,Chinatown
7,Dorchester
8,Downtown
9,East Boston


In [28]:
#create a function to handle TimeOuts from Geocoder
from geopy.exc import GeocoderTimedOut
locator = Nominatim(user_agent = "bostonagent")
def do_geocode(address):
    try:
        return locator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [29]:
neighborhoods = df.values.tolist()

latitude = []
longitude = []
for neighborhood in neighborhoods:
    print(neighborhood)
    coord = do_geocode('{}, Boston, Massachusetts'.format(neighborhood))
    #check to make sure all latitude and longitude values are present, handles the case where Nominatim returns None
    if (coord == None):
        latitude.append('0')
        longitude.append('0')
    else:
        latitude.append(coord.latitude)
        longitude.append(coord.longitude)

#add coordinates column to dataframe
df['Latitude'] = latitude
df['Longitude'] = longitude

['Allston']
['Back Bay']
['Bay Village']
['Beacon Hill']
['Brighton']
['Charlestown']
['Chinatown']
['Dorchester']
['Downtown']
['East Boston']
['Fenway Kenmore']
['Hyde Park']
['Jamaica Plain']
['Mattapan']
['Mission Hill']
['North End']
['Roslindale']
['Roxbury']
['South Boston']
['South End']
['West End']
['West Roxbury']


In [30]:
df

,Neighborhood,Latitude,Longitude
0,Allston,42.355434,-71.132127
1,Back Bay,42.350707,-71.079730
2,Bay Village,42.350011,-71.066948
3,Beacon Hill,42.358708,-71.067829
4,Brighton,42.350097,-71.156442
5,Charlestown,42.377875,-71.061996
6,Chinatown,42.351329,-71.062623
7,Dorchester,42.297320,-71.074495
8,Downtown,42.362918,-71.068737
9,East Boston,42.375097,-71.039217


In [31]:
address = 'Boston, Massachusetts'

#Get lat/long values for Boston and print them
location = locator.geocode(address)
lat = location.latitude
long = location.longitude
print('The geograpical coordinates of Boston are {}, {}.'.format(lat, long))

The geograpical coordinates of Boston are 42.3602534, -71.0582912.


In [32]:
boston_map = folium.Map(location=[lat, long], zoom_start=10)

# add markers to map for each neighborhood
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='cadetblue',
        fill=True,
        fill_color='#29BBB9',
        fill_opacity=0.7,
        parse_html=False).add_to(boston_map)  
    
boston_map

In [33]:
CLIENT_ID = 'GPEZ2BHSQDPGBGF2NFK3QT2PZRFIJJS015OE1XMUICVQM1HK' # your Foursquare ID
CLIENT_SECRET = 'RO4RBQEB2YOIN13PUIPLFCHF4XN1YU40ZVPCMITB2Y0D1JEA' # your Foursquare Secret
VERSION = '20191227'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GPEZ2BHSQDPGBGF2NFK3QT2PZRFIJJS015OE1XMUICVQM1HK
CLIENT_SECRET:RO4RBQEB2YOIN13PUIPLFCHF4XN1YU40ZVPCMITB2Y0D1JEA


In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4d4b7105d754a06374d81259'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
      
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
LIMIT = 100
radius = 1000
boston_venues = getNearbyVenues(names=df['Neighborhood'],latitudes=df['Latitude'],longitudes=df['Longitude'], radius=radius)

Allston
Back Bay
Bay Village
Beacon Hill
Brighton
Charlestown
Chinatown
Dorchester
Downtown
East Boston
Fenway Kenmore
Hyde Park
Jamaica Plain
Mattapan
Mission Hill
North End
Roslindale
Roxbury
South Boston
South End
West End
West Roxbury


In [38]:
print(boston_venues.shape)
boston_venues

(1356, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allston,42.355434,-71.132127,Lulu's Allston,42.355068,-71.134107,Comfort Food Restaurant
1,Allston,42.355434,-71.132127,Kaju Tofu House,42.354329,-71.132374,Korean Restaurant
2,Allston,42.355434,-71.132127,BonChon Chicken,42.353105,-71.130921,Fried Chicken Joint
3,Allston,42.355434,-71.132127,Mala Restaurant,42.352960,-71.131033,Chinese Restaurant
4,Allston,42.355434,-71.132127,Fish Market Sushi Bar,42.353039,-71.132975,Sushi Restaurant
...,...,...,...,...,...,...,...
1351,West Roxbury,42.279265,-71.149497,The Squealing Pig,42.285914,-71.155870,American Restaurant
1352,West Roxbury,42.279265,-71.149497,Local 338,42.287442,-71.149298,Bagel Shop
1353,West Roxbury,42.279265,-71.149497,Christo's Pizza,42.287414,-71.150758,Pizza Place
1354,West Roxbury,42.279265,-71.149497,Nick's Pizza and Seafoods,42.276255,-71.138831,Pizza Place


In [39]:
print('There are {} unique categories.'.format(len(boston_venues['Venue Category'].unique())))
boston_venues.groupby('Neighborhood').count()

There are 75 unique categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allston,49,49,49,49,49,49
Back Bay,100,100,100,100,100,100
Bay Village,100,100,100,100,100,100
Beacon Hill,100,100,100,100,100,100
Brighton,57,57,57,57,57,57
Charlestown,40,40,40,40,40,40
Chinatown,100,100,100,100,100,100
Dorchester,18,18,18,18,18,18
Downtown,100,100,100,100,100,100


In [51]:
boston_onehot = pd.get_dummies(boston_venues[['Venue Category']], prefix="", prefix_sep="")

#add neighborhood names back to dataframe
boston_onehot['Neighborhood'] = boston_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1] ]+ list(boston_onehot.columns[:-1])
boston_onehot = boston_onehot[fixed_columns]

boston_onehot.head()

#dropping categories 'Food', 'Food Court', and 'Food Truck' as they are too broad and not specific cuisines. We are also interested only in restaurants
boston_onehot.drop(columns =['Food','Food Court','Food Truck'], axis =1, inplace=True)
print(boston_onehot.columns)

Index(['Neighborhood', 'Afghan Restaurant', 'African Restaurant',
       'American Restaurant', 'Arepa Restaurant', 'Asian Restaurant',
       'BBQ Joint', 'Bagel Shop', 'Bakery', 'Belgian Restaurant', 'Bistro',
       'Brazilian Restaurant', 'Breakfast Spot', 'Burger Joint',
       'Burrito Place', 'Cafeteria', 'Café', 'Cambodian Restaurant',
       'Caribbean Restaurant', 'Chinese Restaurant', 'Comfort Food Restaurant',
       'Cuban Restaurant', 'Deli / Bodega', 'Dim Sum Restaurant', 'Diner',
       'Donut Shop', 'Dumpling Restaurant', 'Eastern European Restaurant',
       'Ethiopian Restaurant', 'Falafel Restaurant', 'Fast Food Restaurant',
       'Fish & Chips Shop', 'French Restaurant', 'Fried Chicken Joint',
       'Gastropub', 'Greek Restaurant', 'Hot Dog Joint', 'Hotpot Restaurant',
       'Indian Restaurant', 'Irish Pub', 'Israeli Restaurant',
       'Italian Restaurant', 'Japanese Restaurant', 'Korean Restaurant',
       'Latin American Restaurant', 'Mac & Cheese Joint',
   

In [52]:
boston_grouped = boston_onehot.groupby('Neighborhood').mean().reset_index()
boston_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,...,Spanish Restaurant,Steakhouse,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,Allston,0.020408,0.000000,0.020408,0.00,0.020408,0.000000,0.000000,0.061224,0.00,...,0.00,0.00000,0.040816,0.00,0.020408,0.000,0.040816,0.020408,0.040816,0.000000
1,Back Bay,0.000000,0.000000,0.130000,0.00,0.030000,0.000000,0.010000,0.030000,0.00,...,0.00,0.04000,0.020000,0.00,0.000000,0.000,0.010000,0.000000,0.030000,0.000000
2,Bay Village,0.000000,0.000000,0.080000,0.00,0.050000,0.000000,0.000000,0.090000,0.00,...,0.00,0.04000,0.050000,0.01,0.000000,0.000,0.000000,0.010000,0.000000,0.000000
3,Beacon Hill,0.000000,0.000000,0.080000,0.00,0.010000,0.000000,0.020000,0.040000,0.01,...,0.00,0.04000,0.040000,0.00,0.000000,0.000,0.000000,0.010000,0.000000,0.000000
4,Brighton,0.000000,0.000000,0.017544,0.00,0.017544,0.000000,0.000000,0.052632,0.00,...,0.00,0.00000,0.052632,0.00,0.000000,0.000,0.035088,0.017544,0.000000,0.017544
5,Charlestown,0.000000,0.000000,0.025000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.00,0.00000,0.000000,0.00,0.000000,0.025,0.025000,0.000000,0.000000,0.000000
6,Chinatown,0.000000,0.000000,0.040000,0.00,0.070000,0.000000,0.000000,0.090000,0.00,...,0.00,0.04000,0.050000,0.01,0.000000,0.000,0.000000,0.020000,0.000000,0.000000
7,Dorchester,0.000000,0.000000,0.000000,0.00,0.000000,0.055556,0.000000,0.000000,0.00,...,0.00,0.00000,0.000000,0.00,0.000000,0.000,0.000000,0.055556,0.000000,0.000000
8,Downtown,0.000000,0.000000,0.090000,0.00,0.010000,0.010000,0.020000,0.030000,0.01,...,0.00,0.02000,0.020000,0.00,0.000000,0.000,0.030000,0.010000,0.010000,0.000000
9,East Boston,0.000000,0.000000,0.025000,0.00,0.000000,0.000000,0.000000,0.025000,0.00,...,0.00,0.00000,0.000000,0.00,0.025000,0.000,0.000000,0.000000,0.025000,0.000000


In [53]:
num_top_venues = 10

for hood in boston_grouped['Neighborhood']:
    print("----Top Ten Venues in "+hood+"----")
    temp = boston_grouped[boston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['Venue','Frequency']
    temp = temp.iloc[1:]
    temp['Frequency'] = temp['Frequency'].astype(float)
    temp = temp.round({'Frequency': 2})
    print(temp.sort_values('Frequency', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Top Ten Venues in Allston----
                   Venue  Frequency
0      Korean Restaurant       0.10
1                 Bakery       0.06
2            Pizza Place       0.06
3    Japanese Restaurant       0.04
4  Vietnamese Restaurant       0.04
5                  Diner       0.04
6        Thai Restaurant       0.04
7    Fried Chicken Joint       0.04
8       Sushi Restaurant       0.04
9              Gastropub       0.04


----Top Ten Venues in Back Bay----
                     Venue  Frequency
0      American Restaurant       0.13
1       Italian Restaurant       0.11
2       Seafood Restaurant       0.08
3           Sandwich Place       0.07
4  New American Restaurant       0.05
5        French Restaurant       0.04
6               Steakhouse       0.04
7       Mexican Restaurant       0.04
8             Burger Joint       0.03
9         Asian Restaurant       0.03


----Top Ten Venues in Bay Village----
                 Venue  Frequency
0   Italian Restaurant       0.09
1      

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = boston_grouped['Neighborhood']

for ind in np.arange(boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,Korean Restaurant,Bakery,Pizza Place,Vietnamese Restaurant,Thai Restaurant,Japanese Restaurant,Café,Sushi Restaurant,Chinese Restaurant,Diner
1,Back Bay,American Restaurant,Italian Restaurant,Seafood Restaurant,Sandwich Place,New American Restaurant,Steakhouse,Mexican Restaurant,French Restaurant,Café,Burger Joint
2,Bay Village,Bakery,Italian Restaurant,American Restaurant,Seafood Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Sushi Restaurant,Asian Restaurant,Mexican Restaurant
3,Beacon Hill,Italian Restaurant,American Restaurant,Seafood Restaurant,Sandwich Place,Pizza Place,Restaurant,New American Restaurant,Sushi Restaurant,Steakhouse,French Restaurant
4,Brighton,Pizza Place,Chinese Restaurant,Café,Greek Restaurant,Donut Shop,Deli / Bodega,Sushi Restaurant,Bakery,Fast Food Restaurant,Sandwich Place


In [56]:
# set number of clusters
kclusters = 3

boston_grouped_clustering = boston_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 2, 1, 1, 2, 1, 2, 2], dtype=int32)

In [46]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

boston_merged = df

# merge boston_grouped with df to add latitude/longitude for each neighborhood
boston_merged = boston_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

boston_merged # check the last columns!

,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston,42.355434,-71.132127,Korean Restaurant,Bakery,Pizza Place,Café,Thai Restaurant,Diner,Japanese Restaurant,Sushi Restaurant,Fried Chicken Joint,Gastropub
1,Back Bay,42.350707,-71.079730,American Restaurant,Italian Restaurant,Seafood Restaurant,Sandwich Place,New American Restaurant,Mexican Restaurant,Steakhouse,French Restaurant,Vietnamese Restaurant,Fast Food Restaurant
2,Bay Village,42.350011,-71.066948,Italian Restaurant,Bakery,Seafood Restaurant,American Restaurant,Chinese Restaurant,Pizza Place,Sandwich Place,Sushi Restaurant,Asian Restaurant,Mexican Restaurant
3,Beacon Hill,42.358708,-71.067829,Italian Restaurant,American Restaurant,Seafood Restaurant,Sandwich Place,Pizza Place,Bakery,New American Restaurant,French Restaurant,Sushi Restaurant,Steakhouse
4,Brighton,42.350097,-71.156442,Pizza Place,Chinese Restaurant,Café,Sushi Restaurant,Donut Shop,Deli / Bodega,Greek Restaurant,Bakery,Sandwich Place,Thai Restaurant
5,Charlestown,42.377875,-71.061996,Café,Pizza Place,Donut Shop,Gastropub,Sandwich Place,Deli / Bodega,Food,Moroccan Restaurant,New American Restaurant,Italian Restaurant
6,Chinatown,42.351329,-71.062623,Chinese Restaurant,Sandwich Place,Bakery,Asian Restaurant,Sushi Restaurant,Seafood Restaurant,Steakhouse,Italian Restaurant,American Restaurant,French Restaurant
7,Dorchester,42.297320,-71.074495,Pizza Place,Chinese Restaurant,Café,Sandwich Place,Food,Fish & Chips Shop,Fast Food Restaurant,Donut Shop,Diner,Caribbean Restaurant
8,Downtown,42.362918,-71.068737,Italian Restaurant,American Restaurant,Pizza Place,Sandwich Place,Restaurant,Mexican Restaurant,Café,Bakery,Mediterranean Restaurant,Thai Restaurant
9,East Boston,42.375097,-71.039217,Mexican Restaurant,Italian Restaurant,Latin American Restaurant,Sandwich Place,Pizza Place,Food,Café,Donut Shop,Seafood Restaurant,Chinese Restaurant
